In [1]:
import pandas as pd

In [2]:
# get search queries
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')
# get products
product_df = pd.read_csv("WANDS/dataset/product.csv", sep='\t')
# get manually labeled ground truth labels
label_df = pd.read_csv("WANDS/dataset/label.csv", sep='\t')

## Clean the data

In [3]:
query_df.head()

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners


In [54]:
product_df[product_df["product_name"].duplicated()].sort_values(by="product_name")

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
27268,27268,0.8 '' x 0.8 '' porcelain penny round mosaic w...,Floor & Wall Tile,"Home Improvement / Flooring, Walls & Ceiling /...",this product features a glossy fabric look des...,craftsmanshiptype : handmade|material : porcel...,26.0,5.0,25.0
39798,39798,1 - drawer nightstand,Nightstands,Furniture / Bedroom Furniture / Nightstands,keep your side table contemporary with this si...,dswoodtone : espresso wood|overallheight-topto...,1426.0,4.5,908.0
33969,33969,1 1/2 '' diameter round knob,Cabinet and Drawer Knobs,Home Improvement / Hardware / Cabinet Hardware...,"this bubble round knob is drawer pull , cabine...",style : coastal|primarymaterialdetails : steel...,129.0,4.5,85.0
33433,33433,1 1/4 '' diameter mushroom knob,Cabinet and Drawer Knobs,Home Improvement / Hardware / Cabinet Hardware...,the stone mill hardware sidney 1.25 '' round k...,resistancetype : corrosion resistant|knobdiame...,89.0,4.5,55.0
23389,23389,1 3/8 '' diameter mushroom knob,Cabinet and Drawer Knobs,Home Improvement / Hardware / Cabinet Hardware...,"inspired by minimal design elements , this mus...",materialdetails : zinc|overallproductweight:0....,79.0,4.5,55.0
...,...,...,...,...,...,...,...,...,...
7208,7208,wilfredo end table with storage,End Tables,Furniture / Living Room Furniture / Coffee Tab...,if you are looking for an end table that will ...,estimatedtimetosetup:30|estimatedtimetoassembl...,252.0,4.5,205.0
31447,31447,wrington platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,transform your personal space with the classic...,framematerial : manufactured wood|dsprimarypro...,258.0,4.5,168.0
17901,17901,writing desk,Teen Desks,Baby & Kids / Teen Bedroom Furniture / Teen Desks,"the desktop uses a first-grade material , a la...",overallheight-toptobottom:29.9|warrantylength:...,9.0,2.5,8.0
12706,12706,yardmaster2 white portable folding frame proje...,Projector Screens,Furniture / Living Room Furniture / TV Stands ...,yardmaster2 wraithveil dual series portable pr...,overallproductweight:28.9|screentension : tens...,6.0,5.0,5.0


In [55]:
# drop duplicated product names
product_df = product_df.drop_duplicates(subset=["product_name"], keep="first")

In [56]:
# drop products without product class
print(product_df.shape[0])
product_df = product_df.dropna(subset=["product_class"])
print(product_df.shape[0])

42577
39753


In [59]:
# keep only those labels where product id is in product df
print(label_df.shape[0])
label_df = label_df[label_df["product_id"].isin(product_df["product_id"])]
print(label_df.shape[0])

233448
212484


In [65]:
# Intersect query_df query_class and product_df product_class
print(query_df.shape[0])
test = query_df[~query_df["query_class"].isin(product_df["product_class"])]
print(test.shape[0])

480
12


In [68]:
# drop NaN query class
print(query_df.shape[0])
query_df = query_df.dropna(subset=["query_class"])
print(query_df.shape[0])

480
474


In [47]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42994 entries, 0 to 42993
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           42994 non-null  int64  
 1   product_name         42994 non-null  object 
 2   product_class        40142 non-null  object 
 3   category hierarchy   41438 non-null  object 
 4   product_description  36986 non-null  object 
 5   product_features     42994 non-null  object 
 6   rating_count         33542 non-null  float64
 7   average_rating       33542 non-null  float64
 8   review_count         33542 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 3.0+ MB


In [51]:
product_df["product_class"].isna().sum()

2852

In [70]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39753 entries, 0 to 42993
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           39753 non-null  int64  
 1   product_name         39753 non-null  object 
 2   product_class        39753 non-null  object 
 3   category hierarchy   38390 non-null  object 
 4   product_description  34228 non-null  object 
 5   product_features     39753 non-null  object 
 6   rating_count         31409 non-null  float64
 7   average_rating       31409 non-null  float64
 8   review_count         31409 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 3.0+ MB


In [74]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212484 entries, 0 to 233447
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          212484 non-null  int64 
 1   query_id    212484 non-null  int64 
 2   product_id  212484 non-null  int64 
 3   label       212484 non-null  object
dtypes: int64(3), object(1)
memory usage: 8.1+ MB


In [75]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, 0 to 479
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   query_id     474 non-null    int64 
 1   query        474 non-null    object
 2   query_class  474 non-null    object
dtypes: int64(1), object(2)
memory usage: 14.8+ KB


In [77]:
# save cleaned data
query_df.to_csv("data_eval_prepared/query_cleaned.csv", sep='\t', index=False)
product_df.to_csv("data_eval_prepared/product_cleaned.csv", sep='\t', index=False)
label_df.to_csv("data_eval_prepared/label_cleaned.csv", sep='\t', index=False)

In [9]:
label_df["label"].value_counts()

label
Partial       146633
Irrelevant     61201
Exact          25614
Name: count, dtype: int64

In [11]:
label_df["query_id"].value_counts()

query_id
123    4329
446    4326
26     3748
434    3530
131    3479
       ... 
465       2
462       2
348       2
179       2
366       1
Name: count, Length: 480, dtype: int64

In [12]:
import meilisearch

client = meilisearch.Client('http://localhost:7700', 'masterKey')

client.health()

{'status': 'available'}

In [ ]:
client.create_index('items')

In [13]:
client.get_indexes({'limit': 3})

{'results': [<meilisearch.index.Index at 0x1d3d979a700>,
 'offset': 0,
 'limit': 3,
 'total': 3}

In [81]:
hits = client.delete_index("items_eval")
hits

TaskInfo(task_uid=581, index_uid='items_eval', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2023, 12, 24, 0, 11, 37, 616904))